<a href="https://colab.research.google.com/github/ioadeer/bi-bycing-ba-2021/blob/main/recorridos_2020_2021_06_22_2021_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

mpl.rcParams.update( mpl.rcParamsDefault ) 

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id = '1wIUlP0BTE2bk8qpaARZkX4uvA1ucUpin'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('recorridos_2020-2021.zip') 
bicis_recorridos_2020_2021_df = pd.read_csv('recorridos_2020-2021.zip', compression = 'zip', encoding = 'utf-8')
bicis_recorridos_2020_2021_df.head()

,id,duracion_recorrido,id_usuario,id_estacion_origen,nombre_estacion_origen,fecha_origen_recorrido,id_estacion_destino,nombre_estacion_destino,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins
0,0,6563,506009,30,030 - Peña,2020-01-01 00:02:50,30.0,030 - Peña,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,1,jan,1,wed,3,0,2
1,0,1150,116847,206,159 - Fitz Roy & Gorriti,2020-01-01 00:04:19,207.0,123 - Armenia,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,1,jan,1,wed,3,0,4
2,0,383,259973,308,251 - Solar de la Abadía,2020-01-01 00:05:23,289.0,255 - BARRANCAS DE BELGRANO,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,1,jan,1,wed,3,0,5
3,0,72,354594,92,092 - Salcedo,2020-01-01 00:05:42,92.0,092 - Salcedo,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,1,jan,1,wed,3,0,5
4,0,769,222181,41,041 - Urquiza y Rondeau,2020-01-01 00:07:30,41.0,041 - Urquiza y Rondeau,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,1,jan,1,wed,3,0,7


In [7]:
columns_to_drop = ['id_estacion_origen', 'id_estacion_destino']
cleaned = bicis_recorridos_2020_2021_df.drop(columns_to_drop, axis= 1)
cleaned.head()

,id,duracion_recorrido,id_usuario,nombre_estacion_origen,fecha_origen_recorrido,nombre_estacion_destino,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins
0,0,6563,506009,030 - Peña,2020-01-01 00:02:50,030 - Peña,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,1,jan,1,wed,3,0,2
1,0,1150,116847,159 - Fitz Roy & Gorriti,2020-01-01 00:04:19,123 - Armenia,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,1,jan,1,wed,3,0,4
2,0,383,259973,251 - Solar de la Abadía,2020-01-01 00:05:23,255 - BARRANCAS DE BELGRANO,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,1,jan,1,wed,3,0,5
3,0,72,354594,092 - Salcedo,2020-01-01 00:05:42,092 - Salcedo,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,1,jan,1,wed,3,0,5
4,0,769,222181,041 - Urquiza y Rondeau,2020-01-01 00:07:30,041 - Urquiza y Rondeau,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,1,jan,1,wed,3,0,7


In [8]:
aux_list_2d_inicio = pd.DataFrame(cleaned['nombre_estacion_origen'].str.split('-').tolist(), columns =['id_estacion_origen','nombre_estacion_origen'])
aux_list_2d_inicio['nombre_estacion_origen'] = aux_list_2d_inicio['nombre_estacion_origen'].str.lower()
aux_list_2d_inicio


,id_estacion_origen,nombre_estacion_origen
0,030,peña
1,159,fitz roy & gorriti
2,251,solar de la abadía
3,092,salcedo
4,041,urquiza y rondeau
...,...,...
3331666,129,velasco
3331667,117,humberto 1°
3331668,252,tamborini
3331669,278,donato alvarez


In [10]:
bicis_recorridos_2020_2021_df['nombre_estacion_destino'] = bicis_recorridos_2020_2021_df['nombre_estacion_destino'].replace(np.nan, '0 - null')
bicis_recorridos_2020_2021_df['nombre_estacion_destino'].isna().sum()
aux_list_2d_fin = pd.DataFrame(bicis_recorridos_2020_2021_df['nombre_estacion_destino'].str.split('-').tolist(), columns =['id_estacion_destino','nombre_estacion_destino'])
aux_list_2d_fin['nombre_estacion_destino'] = aux_list_2d_fin['nombre_estacion_destino'].str.lower()
aux_list_2d_fin

,id_estacion_destino,nombre_estacion_destino
0,030,peña
1,123,armenia
2,255,barrancas de belgrano
3,092,salcedo
4,041,urquiza y rondeau
...,...,...
3331666,158,villarroel
3331667,147,constitución
3331668,277,coghlan
3331669,349,plaza saenz peña


In [11]:
cleaned = cleaned.drop(['nombre_estacion_destino','nombre_estacion_origen'], axis= 1)
test = pd.concat([cleaned,aux_list_2d_fin, aux_list_2d_inicio ], axis= 1)
test.head()

,id,duracion_recorrido,id_usuario,fecha_origen_recorrido,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins,id_estacion_destino,nombre_estacion_destino,id_estacion_origen,nombre_estacion_origen
0,0,6563,506009,2020-01-01 00:02:50,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,1,jan,1,wed,3,0,2,030,peña,030,peña
1,0,1150,116847,2020-01-01 00:04:19,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,1,jan,1,wed,3,0,4,123,armenia,159,fitz roy & gorriti
2,0,383,259973,2020-01-01 00:05:23,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,1,jan,1,wed,3,0,5,255,barrancas de belgrano,251,solar de la abadía
3,0,72,354594,2020-01-01 00:05:42,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,1,jan,1,wed,3,0,5,092,salcedo,092,salcedo
4,0,769,222181,2020-01-01 00:07:30,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,1,jan,1,wed,3,0,7,041,urquiza y rondeau,041,urquiza y rondeau


In [17]:
formato = '%Y%m%d %H:%M:%S'
test['start_datetime'] = pd.to_datetime(
    test['fecha_origen_recorrido'], 
    format = formato
)
test['start_date'] = test['start_datetime'].dt.strftime('%Y-%m-%d')

In [18]:
test.head()

,id,duracion_recorrido,id_usuario,fecha_origen_recorrido,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins,id_estacion_destino,nombre_estacion_destino,id_estacion_origen,nombre_estacion_origen,start_date
0,0,6563,506009,2020-01-01 00:02:50,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,1,jan,1,wed,3,0,2,030,peña,030,peña,2020-01-01
1,0,1150,116847,2020-01-01 00:04:19,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,1,jan,1,wed,3,0,4,123,armenia,159,fitz roy & gorriti,2020-01-01
2,0,383,259973,2020-01-01 00:05:23,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,1,jan,1,wed,3,0,5,255,barrancas de belgrano,251,solar de la abadía,2020-01-01
3,0,72,354594,2020-01-01 00:05:42,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,1,jan,1,wed,3,0,5,092,salcedo,092,salcedo,2020-01-01
4,0,769,222181,2020-01-01 00:07:30,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,1,jan,1,wed,3,0,7,041,urquiza y rondeau,041,urquiza y rondeau,2020-01-01


In [22]:
dias_habiles = [1,2,3,4,5]
     

test_dia_habil_df = test.assign(dia_habil=[1 if dia in dias_habiles else 0 for dia in test['wdia']])

In [30]:
test_dia_habil_df

,id,duracion_recorrido,id_usuario,fecha_origen_recorrido,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins,id_estacion_destino,nombre_estacion_destino,id_estacion_origen,nombre_estacion_origen,start_date,dia_habil
0,0,6563,506009,2020-01-01 00:02:50,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,1,jan,1,wed,3,0,2,030,peña,030,peña,2020-01-01,1
1,0,1150,116847,2020-01-01 00:04:19,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,1,jan,1,wed,3,0,4,123,armenia,159,fitz roy & gorriti,2020-01-01,1
2,0,383,259973,2020-01-01 00:05:23,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,1,jan,1,wed,3,0,5,255,barrancas de belgrano,251,solar de la abadía,2020-01-01,1
3,0,72,354594,2020-01-01 00:05:42,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,1,jan,1,wed,3,0,5,092,salcedo,092,salcedo,2020-01-01,1
4,0,769,222181,2020-01-01 00:07:30,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,1,jan,1,wed,3,0,7,041,urquiza y rondeau,041,urquiza y rondeau,2020-01-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331666,11204919,591,35926,2021-06-02 16:09:32,2021-06-02 16:19:23,2021-06-02 16:09:32,2021,6,jun,153,wed,3,16,9,158,villarroel,129,velasco,2021-06-02,1
3331667,11204920,400,713938,2021-06-02 16:09:34,2021-06-02 16:16:14,2021-06-02 16:09:34,2021,6,jun,153,wed,3,16,9,147,constitución,117,humberto 1°,2021-06-02,1
3331668,11204928,299,690073,2021-06-02 16:10:08,2021-06-02 16:15:07,2021-06-02 16:10:08,2021,6,jun,153,wed,3,16,10,277,coghlan,252,tamborini,2021-06-02,1
3331669,11204938,406,348428,2021-06-02 16:11:01,2021-06-02 16:17:47,2021-06-02 16:11:01,2021,6,jun,153,wed,3,16,11,349,plaza saenz peña,278,donato alvarez,2021-06-02,1


In [ ]:
from google.colab import drive
drive.mount('/drive')

In [13]:
# Fetch weather from drive
id = '14BpBfjQ3IrfeK8CajCWrEdfPg7S0eqc8'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('weather-data-ba.csv') 
weather_data_df = pd.read_csv('weather-data-ba.csv')
weather_data_df.head()
weather_data_df = weather_data_df.filter(
    ['DATE','PRCP','TAVG']
)
weather_data_df.isnull().sum( axis = 0 )
weather_data_df['PRCP'] = weather_data_df['PRCP'].replace( np.nan, 0 )
weather_data_df.head()

,DATE,PRCP,TAVG
0,2017-01-01,0.0,30.3
1,2017-01-02,0.0,25.6
2,2017-01-03,0.0,26.2
3,2017-01-04,0.0,26.2
4,2017-01-05,0.0,25.1


In [31]:
weather_data_df

,DATE,PRCP,TAVG
0,2017-01-01,0.0,30.3
1,2017-01-02,0.0,25.6
2,2017-01-03,0.0,26.2
3,2017-01-04,0.0,26.2
4,2017-01-05,0.0,25.1
...,...,...,...
1608,2021-05-30,0.0,10.7
1609,2021-05-31,0.0,14.4
1610,2021-06-01,0.0,16.3
1611,2021-06-02,0.0,14.6


In [29]:
recorridos_latest_df = pd.merge(left=test_dia_habil_df, right=weather_data_df, left_on='start_date', right_on='DATE')
recorridos_latest_df

,id,duracion_recorrido,id_usuario,fecha_origen_recorrido,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins,id_estacion_destino,nombre_estacion_destino,id_estacion_origen,nombre_estacion_origen,start_date,dia_habil,DATE,PRCP,TAVG
0,0,6563,506009,2020-01-01 00:02:50,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,1,jan,1,wed,3,0,2,030,peña,030,peña,2020-01-01,1,2020-01-01,0.0,22.8
1,0,1150,116847,2020-01-01 00:04:19,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,1,jan,1,wed,3,0,4,123,armenia,159,fitz roy & gorriti,2020-01-01,1,2020-01-01,0.0,22.8
2,0,383,259973,2020-01-01 00:05:23,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,1,jan,1,wed,3,0,5,255,barrancas de belgrano,251,solar de la abadía,2020-01-01,1,2020-01-01,0.0,22.8
3,0,72,354594,2020-01-01 00:05:42,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,1,jan,1,wed,3,0,5,092,salcedo,092,salcedo,2020-01-01,1,2020-01-01,0.0,22.8
4,0,769,222181,2020-01-01 00:07:30,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,1,jan,1,wed,3,0,7,041,urquiza y rondeau,041,urquiza y rondeau,2020-01-01,1,2020-01-01,0.0,22.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310658,11204919,591,35926,2021-06-02 16:09:32,2021-06-02 16:19:23,2021-06-02 16:09:32,2021,6,jun,153,wed,3,16,9,158,villarroel,129,velasco,2021-06-02,1,2021-06-02,0.0,14.6
3310659,11204920,400,713938,2021-06-02 16:09:34,2021-06-02 16:16:14,2021-06-02 16:09:34,2021,6,jun,153,wed,3,16,9,147,constitución,117,humberto 1°,2021-06-02,1,2021-06-02,0.0,14.6
3310660,11204928,299,690073,2021-06-02 16:10:08,2021-06-02 16:15:07,2021-06-02 16:10:08,2021,6,jun,153,wed,3,16,10,277,coghlan,252,tamborini,2021-06-02,1,2021-06-02,0.0,14.6
3310661,11204938,406,348428,2021-06-02 16:11:01,2021-06-02 16:17:47,2021-06-02 16:11:01,2021,6,jun,153,wed,3,16,11,349,plaza saenz peña,278,donato alvarez,2021-06-02,1,2021-06-02,0.0,14.6


In [32]:
recorridos_latest_cleaned_df = recorridos_latest_df.drop(['start_date','DATE'], axis= 1)

In [34]:
recorridos_latest_cleaned_df

,id,duracion_recorrido,id_usuario,fecha_origen_recorrido,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins,id_estacion_destino,nombre_estacion_destino,id_estacion_origen,nombre_estacion_origen,dia_habil,PRCP,TAVG
0,0,6563,506009,2020-01-01 00:02:50,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,1,jan,1,wed,3,0,2,030,peña,030,peña,1,0.0,22.8
1,0,1150,116847,2020-01-01 00:04:19,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,1,jan,1,wed,3,0,4,123,armenia,159,fitz roy & gorriti,1,0.0,22.8
2,0,383,259973,2020-01-01 00:05:23,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,1,jan,1,wed,3,0,5,255,barrancas de belgrano,251,solar de la abadía,1,0.0,22.8
3,0,72,354594,2020-01-01 00:05:42,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,1,jan,1,wed,3,0,5,092,salcedo,092,salcedo,1,0.0,22.8
4,0,769,222181,2020-01-01 00:07:30,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,1,jan,1,wed,3,0,7,041,urquiza y rondeau,041,urquiza y rondeau,1,0.0,22.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310658,11204919,591,35926,2021-06-02 16:09:32,2021-06-02 16:19:23,2021-06-02 16:09:32,2021,6,jun,153,wed,3,16,9,158,villarroel,129,velasco,1,0.0,14.6
3310659,11204920,400,713938,2021-06-02 16:09:34,2021-06-02 16:16:14,2021-06-02 16:09:34,2021,6,jun,153,wed,3,16,9,147,constitución,117,humberto 1°,1,0.0,14.6
3310660,11204928,299,690073,2021-06-02 16:10:08,2021-06-02 16:15:07,2021-06-02 16:10:08,2021,6,jun,153,wed,3,16,10,277,coghlan,252,tamborini,1,0.0,14.6
3310661,11204938,406,348428,2021-06-02 16:11:01,2021-06-02 16:17:47,2021-06-02 16:11:01,2021,6,jun,153,wed,3,16,11,349,plaza saenz peña,278,donato alvarez,1,0.0,14.6


In [33]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [35]:
recorridos_latest_cleaned_df.to_csv('/drive/My Drive/Datasets-bicing/recorridos-2020-2021-06-22-2021.csv')